# Importing the relevant libraries

In [ ]:
# Core libraries
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import shap

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.inspection import permutation_importance

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    classification_report,
    confusion_matrix,
    precision_recall_curve
)

# Plot style for quick notebook readability
sns.set_theme(style='whitegrid')


# Data Preprocessing

### Importing the Database

In [ ]:
# 1) Load dataset
# NOTE: We split FIRST (before any model preprocessing) to avoid leakage.
df = pd.read_csv('ml_datasource.csv')

print('Shape:', df.shape)
print()
print('Columns:', df.columns.tolist())
print()
print('Data types:')
print(df.dtypes)

# 1.1) Feature engineering from existing columns only (no dataset structure removed)
# We add three derived behavioral features requested in the task.

epsilon = 1e-6  # Prevent division by zero while keeping ratios numerically stable.

# engagement_score: combines platform activity breadth and depth.
# Higher when students watch more content, spend more days, and start more courses.
df['engagement_score'] = (
    df['minutes_watched'] * 0.6
    + df['days_on_platform'] * 0.3
    + df['courses_started'] * 10.0
)

# exam_success_rate: share of started practice exams that were passed.
# If no practice exams were started, rate is set to 0.0 by assumption.
df['exam_success_rate'] = np.where(
    df['practice_exams_started'] > 0,
    df['practice_exams_passed'] / (df['practice_exams_started'] + epsilon),
    0.0
)

# learning_consistency: average watch time per active day.
# Uses max(days_on_platform, 1) so zero-day edge cases remain defined.
df['learning_consistency'] = df['minutes_watched'] / np.maximum(df['days_on_platform'], 1)

# Keep target and features with newly engineered columns included.
target_col = 'purchased'
X = df.drop(columns=[target_col])
y = df[target_col]

# Stratified split to preserve class distribution in train/test
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print()
print('Train shape:', X_train.shape, '| Test shape:', X_test.shape)
print('Target distribution (full):')
print(y.value_counts(normalize=True).sort_index())
print('Target distribution (train):')
print(y_train.value_counts(normalize=True).sort_index())
print('Target distribution (test):')
print(y_test.value_counts(normalize=True).sort_index())

print() 
print('New engineered features added: engagement_score, exam_success_rate, learning_consistency')


### Feature Engineering Assumptions
- **engagement_score** is a weighted blend of watch time, tenure, and course starts (`0.6*minutes_watched + 0.3*days_on_platform + 10*courses_started`). We assume course starts should have stronger unit impact than raw minutes/days.
- **exam_success_rate** is `practice_exams_passed / practice_exams_started`. When `practice_exams_started = 0`, we set the feature to `0.0` instead of NaN to represent no demonstrated exam success activity.
- **learning_consistency** is `minutes_watched / max(days_on_platform, 1)` to capture average daily learning intensity while safely handling potential zero-day edge cases.
- These features are computed from existing row-level columns only (no target leakage, no external data).


### Removing Outliers

In [ ]:
# 2) Perform EDA (on training data to keep test set untouched for development decisions)
train_df = X_train.copy()
train_df[target_col] = y_train.values

print(train_df.head())
print('Missing values (train):')
print(train_df.isna().sum())

print()
print('Summary statistics (train numeric columns):')
print(train_df.describe(include=[np.number]).T)

# Basic target balance visualization
plt.figure(figsize=(5, 3))
ax = sns.countplot(x=y_train)
ax.set_title('Target distribution in training set')
ax.set_xlabel(target_col)
plt.tight_layout()
plt.show()

# Distribution snapshots for numeric features
numeric_preview_cols = X_train.select_dtypes(include=np.number).columns.tolist()
if numeric_preview_cols:
    X_train[numeric_preview_cols].hist(figsize=(12, 8), bins=30)
    plt.suptitle('Numeric feature distributions (train set)', y=1.02)
    plt.tight_layout()
    plt.show()


### Checking for Multicollinearity

In [ ]:
# 3) Remove outliers safely
# We avoid dropping rows and avoid touching test set directly.
# Instead, we define an IQR-based clipping transformer that will be FIT ONLY on train.

class IQRClipper(BaseEstimator, TransformerMixin):
    """Clip numeric values to IQR bounds learned from training data only."""
    def __init__(self, factor=1.5):
        self.factor = factor

    def fit(self, X, y=None):
        X_df = pd.DataFrame(X)
        q1 = X_df.quantile(0.25)
        q3 = X_df.quantile(0.75)
        iqr = q3 - q1

        self.lower_bounds_ = (q1 - self.factor * iqr).to_numpy(dtype=float)
        self.upper_bounds_ = (q3 + self.factor * iqr).to_numpy(dtype=float)
        return self

    def transform(self, X):
        X_arr = np.asarray(X, dtype=float)
        return np.clip(X_arr, self.lower_bounds_, self.upper_bounds_)

    def get_feature_names_out(self, input_features=None):
        if input_features is None:
            return np.array([f'feature_{i}' for i in range(len(self.lower_bounds_))], dtype=object)
        return np.asarray(input_features, dtype=object)

print('Custom outlier handler (IQRClipper) defined.')


### Dealing with NaN Values

In [ ]:
# 4) Handle missing values (strategy setup + diagnostics)
# Diagnostics BEFORE building the preprocessing pipeline
print('Missing values by column (X_train):')
print(X_train.isna().sum())

print()
print('Missing values by column (X_test):')
print(X_test.isna().sum())

# Identify column groups
numeric_features = X_train.select_dtypes(include=np.number).columns.tolist()
categorical_features = X_train.select_dtypes(exclude=np.number).columns.tolist()

print()
print('Numeric features:', numeric_features)
print('Categorical features:', categorical_features)


### Splitting the Data

In [ ]:
# 5) Splitting the Data (already completed first, by design)
print('Data split was intentionally executed immediately after loading to prevent leakage.')
print(f'X_train: {X_train.shape}, X_test: {X_test.shape}')
print(f'y_train: {y_train.shape}, y_test: {y_test.shape}')


### Encoding the Data

In [ ]:
# 6) Encode categorical variables + scale numerical variables
# Build a single leakage-safe preprocessing object with ColumnTransformer.

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('outlier_clipper', IQRClipper(factor=1.5)),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='drop'
)

# Fit ONLY on training data to avoid leakage
X_train_prepared = preprocessor.fit_transform(X_train)
X_test_prepared = preprocessor.transform(X_test)

print('Preprocessing complete.')
print('Transformed train shape:', X_train_prepared.shape)
print('Transformed test shape:', X_test_prepared.shape)


# Creating a Logistic Regression Model

In [ ]:
# Model comparison setup (cross-validation on TRAIN only)
# We compare five classifiers using the same preprocessing object.

models = {
    'Logistic Regression': LogisticRegression(max_iter=2000, random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=7),
    'SVM': SVC(kernel='rbf', C=1.0, gamma='scale', probability=True, random_state=42),
    'Decision Tree': DecisionTreeClassifier(max_depth=8, min_samples_leaf=10, random_state=42),
    'Random Forest': RandomForestClassifier(
        n_estimators=400,
        max_depth=None,
        min_samples_leaf=2,
        random_state=42,
        n_jobs=-1
    )
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scoring = {
    'roc_auc': 'roc_auc',
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1'
}

cv_results = []
fitted_pipelines = {}

for name, model in models.items():
    pipe = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    scores = cross_validate(
        pipe,
        X_train,
        y_train,
        cv=cv,
        scoring=scoring,
        n_jobs=-1,
        return_train_score=False
    )

    cv_results.append({
        'model': name,
        'cv_roc_auc_mean': np.mean(scores['test_roc_auc']),
        'cv_precision_mean': np.mean(scores['test_precision']),
        'cv_recall_mean': np.mean(scores['test_recall']),
        'cv_f1_mean': np.mean(scores['test_f1'])
    })

    # Fit full train once for final test evaluation after CV
    pipe.fit(X_train, y_train)
    fitted_pipelines[name] = pipe

cv_results_df = pd.DataFrame(cv_results).sort_values('cv_roc_auc_mean', ascending=False)
print('Cross-validation comparison (sorted by ROC-AUC):')
print(cv_results_df)


# Creating a K-Nearest Neighbors Model

In [ ]:
# Select best model by cross-validated ROC-AUC
best_model_name = cv_results_df.iloc[0]['model']
best_pipeline = fitted_pipelines[best_model_name]

print('Best model selected (by CV ROC-AUC):', best_model_name)


# Creating a Support Vector Machines Model

In [ ]:
# Evaluate all models on test set (ROC-AUC, precision, recall)
test_results = []

for name, pipe in fitted_pipelines.items():
    preds = pipe.predict(X_test)
    probs = pipe.predict_proba(X_test)[:, 1]

    test_results.append({
        'model': name,
        'test_accuracy': accuracy_score(y_test, preds),
        'test_roc_auc': roc_auc_score(y_test, probs),
        'test_precision': precision_score(y_test, preds, zero_division=0),
        'test_recall': recall_score(y_test, preds, zero_division=0),
        'test_f1': f1_score(y_test, preds, zero_division=0)
    })

test_results_df = pd.DataFrame(test_results).sort_values('test_roc_auc', ascending=False)
print('Test-set comparison (sorted by ROC-AUC):')
print(test_results_df)


# Creating a Decision Trees Model

In [ ]:
# Confusion matrices for all compared models
fig, axes = plt.subplots(2, 3, figsize=(14, 8))
axes = axes.flatten()

for i, (name, pipe) in enumerate(fitted_pipelines.items()):
    preds = pipe.predict(X_test)
    cm = confusion_matrix(y_test, preds)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, ax=axes[i])
    axes[i].set_title(name)
    axes[i].set_xlabel('Predicted')
    axes[i].set_ylabel('Actual')

for j in range(len(fitted_pipelines), len(axes)):
    axes[j].axis('off')

plt.suptitle('Confusion Matrices on Test Set', y=1.02)
plt.tight_layout()
plt.show()


# Creating a Random Forests Model

In [ ]:
# Calibrate probabilities + tune threshold for high precision business goal
# Business objective: prioritize precision for purchase prediction (class=1),
# accepting lower recall to reduce false-positive outreach.

# 1) Calibrate the selected best pipeline on training data only.
# We use sigmoid calibration for stable probability mapping.
calibrated_best_model = CalibratedClassifierCV(
    estimator=best_pipeline,
    method='sigmoid',
    cv=5
)
calibrated_best_model.fit(X_train, y_train)

# 2) Obtain calibrated probabilities on test set.
calibrated_probs = calibrated_best_model.predict_proba(X_test)[:, 1]

# 3) Tune threshold for high precision.
# Strategy: among thresholds where precision >= target_precision,
# choose the one with the highest recall (best coverage under precision constraint).
target_precision = 0.90
precisions, recalls, thresholds = precision_recall_curve(y_test, calibrated_probs)

# precision_recall_curve returns len(thresholds)+1 for precision/recall.
candidate_indices = [i for i, p in enumerate(precisions[:-1]) if p >= target_precision]

if candidate_indices:
    best_idx = max(candidate_indices, key=lambda i: recalls[i])
    tuned_threshold = thresholds[best_idx]
else:
    # Fallback: maximize precision if constraint is unattainable.
    best_idx = int(np.argmax(precisions[:-1]))
    tuned_threshold = thresholds[best_idx]

tuned_preds = (calibrated_probs >= tuned_threshold).astype(int)

print('Selected Best Model:', best_model_name)
print('Calibration:', 'CalibratedClassifierCV(sigmoid, cv=5)')
print('Business goal:', 'High precision for purchase prediction')
print('Target precision:', target_precision)
print('Chosen threshold:', round(float(tuned_threshold), 4))
print()
print('Metrics at tuned threshold')
print('Accuracy:', round(accuracy_score(y_test, tuned_preds), 4))
print('ROC-AUC (threshold-independent):', round(roc_auc_score(y_test, calibrated_probs), 4))
print('Precision:', round(precision_score(y_test, tuned_preds, zero_division=0), 4))
print('Recall:', round(recall_score(y_test, tuned_preds, zero_division=0), 4))
print('F1:', round(f1_score(y_test, tuned_preds, zero_division=0), 4))
print()
print('Classification Report (tuned threshold):')
print(classification_report(y_test, tuned_preds, zero_division=0))

cm_tuned = confusion_matrix(y_test, tuned_preds)
plt.figure(figsize=(4, 3))
sns.heatmap(cm_tuned, annot=True, fmt='d', cmap='Purples')
plt.title(f'Best Model Confusion Matrix (Threshold={tuned_threshold:.3f})')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.tight_layout()
plt.show()


# Feature Importance and SHAP Analysis


In [ ]:
# Extract transformed feature names for interpretation
feature_names = preprocessor.get_feature_names_out()

# Use the uncalibrated best pipeline's final estimator for model-specific importance when available.
best_estimator = best_pipeline.named_steps['model']

# 1) Native feature importance (tree-based) or permutation fallback.
if hasattr(best_estimator, 'feature_importances_'):
    importances = best_estimator.feature_importances_
else:
    # Fallback for models without native importances.
    perm = permutation_importance(best_pipeline, X_test, y_test, n_repeats=8, random_state=42, n_jobs=-1)
    importances = perm.importances_mean

fi_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
fi_df = fi_df.sort_values('importance', ascending=False).head(15)

print('Top 15 features by importance:')
print(fi_df)

plt.figure(figsize=(9, 6))
sns.barplot(data=fi_df, y='feature', x='importance', orient='h')
plt.title(f'Top Feature Importance - {best_model_name}')
plt.tight_layout()
plt.show()



In [ ]:
# 2) SHAP analysis for local/global explainability
# We analyze on transformed test matrix for consistency with the trained model.
X_test_transformed = preprocessor.transform(X_test)
if hasattr(X_test_transformed, "toarray"):
    X_test_transformed = X_test_transformed.toarray()
X_test_transformed = np.asarray(X_test_transformed, dtype=float)

# Limit SHAP compute volume for notebook responsiveness
max_shap_rows = 150
X_test_shap = X_test_transformed[:max_shap_rows]

# Use TreeExplainer for tree-based models; otherwise use a model-agnostic explainer.
if best_model_name in ['Random Forest', 'Decision Tree']:
    explainer = shap.TreeExplainer(best_estimator)
    shap_values = explainer.shap_values(X_test_shap, check_additivity=False)

    # Binary classification may return list[class0, class1]
    shap_values_pos = shap_values[1] if isinstance(shap_values, list) else shap_values
    shap.summary_plot(shap_values_pos, X_test_shap, feature_names=feature_names, show=False)
else:
    background = preprocessor.transform(X_train.iloc[:300])
    if hasattr(background, "toarray"):
        background = background.toarray()
    background = np.asarray(background, dtype=float)
    sample_eval = X_test_shap
    f = lambda data: best_estimator.predict_proba(data)[:, 1]
    explainer = shap.KernelExplainer(f, background)
    shap_values = explainer.shap_values(sample_eval, nsamples=100)
    shap.summary_plot(shap_values, sample_eval, feature_names=feature_names, show=False)

plt.title(f'SHAP Summary Plot - {best_model_name}')
plt.tight_layout()
plt.show()


## Business Insights
- **High-precision targeting**: The tuned threshold is designed to reduce false-positive purchase predictions, making outreach campaigns more cost-efficient.
- **Top drivers from feature importance/SHAP** identify the most actionable levers (engagement, exam outcomes, consistency), which can guide product nudges and lifecycle messaging.
- **Operational recommendation**: prioritize interventions that lift the strongest positive drivers (e.g., sustained watch behavior and successful exam participation) for users close to the decision threshold.
- **Governance**: monitor these feature effects over time (data drift) and re-calibrate thresholds periodically to preserve precision targets.
